In [1]:
import numpy as np
import pandas as pd
import random
import cvxopt
from cvxopt import matrix, solvers
from cvxopt.modeling import variable
from tqdm.notebook import tqdm
from sklearn.svm import SVC
from libsvm.svmutil import *
from datetime import datetime

In [2]:
SUBSET = 2000 # subset for tuning
N_FOLDS = 10 # k-Fold cross validation
NUM_FEATURES = 25 # <=25 for the given data
MODE = 'LIBSVM' # 'LIBSVM', 'CVXOPT', 'SSMO'
C1,C2 = 1,8 # Classes (0-9)
KERNEL = 'LINEAR' # 'LINEAR', 'POLY', 'RBF'
GAMMA = 0.001 # Gamma for RBF Kernel
C = 3.0 # C for L1 regularization
POWER = 2 # Power in Polynomial kernel
COEFF = 4 # Coefficient in Polynomial kernel
MAX_PASSES = 5 # Number of passes in SSMO
TOLERANCE = 1e-3 # Tolerance for SSMO

In [3]:
def getData(C1,C2):
    #df = pd.read_csv('2019EE10577.csv',header=None)
    #df = df[(df[25] == C1) | (df[25] == C2)]
    df = pd.read_csv('train_set.csv',header=None)
    dataset = []
    for i,row in df.iterrows():
        xi = []
        for j in range(0,NUM_FEATURES+1):
            xi.append(row[j])
#         if(row[25]==C1):
#             xi.append(-1.0)
#         else:
#             xi.append(1.0)
        dataset.append(xi)
    return dataset[:SUBSET]

In [4]:
def cross_validation_split(dataset):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / N_FOLDS)
    for i in range(N_FOLDS):
        fold = list()
        while len(fold) < fold_size:
            index = random.randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

In [5]:
dataset = getData(C1,C2)

In [6]:
dataset_split = cross_validation_split(dataset)

In [7]:
class SVM:
    def __init__(self,mode='CVXOPT',kernel='LINEAR',C = 3.0,gamma = 0.001, power = 2, coeff = 4, max_passes = 5, tol=1e-3):
        self.mode = mode
        self.kernel = kernel
        self.C = C
        self.gamma = gamma
        self.power = power
        self.coeff = coeff
        self.max_passes = max_passes
        self.tol = tol
    
    def fit(self,X,y):
        X = np.array(X)
        y = np.array(y)
        num_samples,num_features = X.shape
        K = np.zeros((num_samples,num_samples))
        
        for i in (range(num_samples)):
            for j in range(num_samples):
                if self.kernel == 'LINEAR':
                    K[i][j] = np.dot(X[i],np.transpose(X[j]))
                elif self.kernel == 'POLY':
                    K[i][j] = (np.dot(X[i],np.transpose(X[j])) + self.coeff) ** self.power
                elif self.kernel == 'RBF':
                    K[i][j] = np.exp(-1 * self.gamma*np.sum(np.square(X[i]-X[j])))
        
        self.K = K
        
        if self.mode == 'CVXOPT':
            H = np.zeros((num_samples,num_samples))
            for i in (range(num_samples)):
                for j in range(num_samples):
                    H[i][j] = y[i]*y[j] * K[i][j]
            P = matrix(H)
            q = matrix(np.ones(num_samples) * -1)
            G = matrix(np.vstack(((np.identity(num_samples) * -1),np.identity(num_samples))))
            h = matrix(np.hstack((np.zeros(num_samples),np.ones(num_samples)*self.C)))
            A = matrix(y,(1,num_samples))
            b = matrix(0.0)

            solvers.options['show_progress'] = False
            soln = solvers.qp(P,q,G,h,A,b)
            alpha = np.array(soln['x']) 

            self.sup_idx = np.where(alpha>1e-5)[0]
            self.ind =  np.arange(len(alpha))[self.sup_idx]

            self.sup_x = X[self.sup_idx,:]
            self.sup_y = y[self.sup_idx]
            self.alpha = alpha[self.sup_idx]
            self.b = self.sup_y[0]

            for i in range(len(self.alpha)):
                if self.kernel == 'LINEAR':
                    temp = np.dot(self.sup_x[i],np.transpose(self.sup_x[0]))
                elif self.kernel == 'POLY':
                    temp = (np.dot(self.sup_x[i],np.transpose(self.sup_x[0]))+self.coeff)**self.power
                elif self.kernel == 'RBF':
                    temp = np.exp(-1 * self.gamma*np.sum(np.square(self.sup_x[i]-self.sup_x[0])))
                self.b -= self.alpha[i] * self.sup_y[i] * temp
        
        elif self.mode == 'SSMO':
            alpha = np.zeros((num_samples))
            alpha_old = np.zeros((num_samples))
            E = np.zeros((num_samples))
            b = 0
            passes = 0
            while(passes<self.max_passes):
                num_changed_alpha = 0
                for i in range(num_samples):
                    E[i] = np.sum(alpha * y * self.K[:,i]) + b - y[i]

                    if (y[i] * E[i] < -self.tol and alpha[i]<self.C) or (y[i] * E[i] > self.tol and alpha[i]>0):
                        j = random.randint(0,num_samples-1)
                        while j == i:
                            j = random.randint(0,num_samples-1)

                        E[j] = np.sum(alpha * y * self.K[:,j]) + b - y[j]
                        alpha_old[i] = alpha[i]
                        alpha_old[j] = alpha[j]

                        if y[i] != y[j]:
                            L = max(0,alpha[j] - alpha[i])
                            H = min(self.C,self.C+alpha[j] - alpha[i])
                        else:
                            L = max(0,alpha[i]+alpha[j] - self.C)
                            H = min(self.C,alpha[i] + alpha[j])

                        if L==H:
                            continue

                        eta = 2* self.K[i][j] - self.K[i][i] - self.K[j][j]

                        if eta>=0:
                            continue

                        alpha[j] = alpha[j] - y[j] * (E[i] - E[j])/eta

                        if alpha[j]>H:
                            alpha[j] = H
                        elif alpha[j]<L:
                            alpha[j] = L

                        if abs(alpha[j] - alpha_old[j]) < 1e-5:
                            continue

                        alpha[i] = alpha[i] + y[i]* y[j] * (alpha_old[j] - alpha[j])

                        b1 = b - E[i] - y[i] * (alpha[i] - alpha_old[i]) * self.K[i][i] \
                            - y[j] * (alpha[j] - alpha_old[j]) * self.K[i][j]
                        b2 = b - E[j] - y[i] * (alpha[i] - alpha_old[i]) * self.K[i][j] \
                            - y[j] * (alpha[j] - alpha_old[j]) * self.K[j][j] 

                        if 0<alpha[i] and alpha[i] <self.C:
                            b = b1
                        elif 0<alpha[j] and alpha[j] < self.C:
                            b = b2
                        else:
                            b = (b1+b2)/2

                        num_changed_alpha +=1

                if num_changed_alpha == 0:
                    passes+=1
                else:
                    passes=0

            self.b = b
            self.sup_idx = np.where(alpha>0)[0]
            self.sup_x = X[self.sup_idx,:]
            self.sup_y = y[self.sup_idx]
            self.alpha = alpha[self.sup_idx]
            self.w = np.dot(alpha*y,X).T
        
    def predict(self,X):
        preds = []
        for x in X:
            pred = 0
            for i in range(len(self.alpha)):
                if self.kernel == 'LINEAR':
                    temp = np.dot(self.sup_x[i],np.transpose(x))
                elif self.kernel == 'POLY':
                    temp = (np.dot(self.sup_x[i],np.transpose(x)) + self.coeff) ** self.power
                elif self.kernel == 'RBF':
                    temp = np.exp(-1 * self.gamma *np.sum(np.square(self.sup_x[i]-x)))
                pred += self.alpha[i] * self.sup_y[i] * temp
            pred += self.b
            if pred>=0:
                preds.append(1.0)
            else:
                preds.append(-1.0)
        return np.array(preds)

In [8]:
def getAccuracy(preds,testY):
    correct = 0
    for i in range(len(testY)):
        if preds[i] == testY[i]:
            correct +=1
    return correct/len(testY) * 100

In [9]:
def evaluate(dataset_split):
    train_acc_list = []
    test_acc_list = []
    for fold in (dataset_split):
        trainDS = list(dataset_split)
        trainDS.remove(fold)
        trainDS = sum(trainDS,[])
        testDS = list()
        
        for row in fold:
            rowCopy = list(row)
            testDS.append(rowCopy)
        
        trainX = [train[:-1] for train in trainDS]
        trainY = [train[-1] for train in trainDS]
        
        testX = [test[:-1] for test in testDS]
        testY = [test[-1] for test in testDS]
        
        if MODE == 'LIBSVM':
            prob = svm_problem(trainY,trainX)
            param = svm_parameter()
            kernel_string_map = {
                'LINEAR': LINEAR,
                'POLY': POLY,
                'RBF': RBF
            }
            param.kernel_type = kernel_string_map[KERNEL]
            param.C = C
            param.gamma = GAMMA
            param.degree = POWER
            param.coef0 = COEFF
            m = svm_train(prob,param,'-q')
            t_label, t_acc, t_val = svm_predict(trainY, trainX, m,'-q')
            p_label, p_acc, p_val = svm_predict(testY, testX, m,'-q')
            train_acc_list.append(t_acc[0])
            test_acc_list.append(p_acc[0])
        else:
            cf = SVM(mode=MODE,kernel=KERNEL,gamma=GAMMA,C=C,power=POWER,coeff=COEFF,max_passes=MAX_PASSES,tol=TOLERANCE)
            cf.fit(trainX,trainY)
            
            train_preds = cf.predict(trainX)
            test_preds = cf.predict(testX)
            
            train_acc = getAccuracy(train_preds,trainY)
            test_acc = getAccuracy(test_preds,testY)
            test_acc_list.append(test_acc)
        
    return np.mean(train_acc_list),np.mean(test_acc_list)

In [10]:
evaluate(dataset_split)

(96.8111111111111, 87.05)

In [17]:
for C in np.arange(0.5,2,0.1):
    train_acc,test_acc = evaluate(dataset_split)
    print(C, train_acc,test_acc)

0.5 98.74551971326164 97.90322580645162
0.6 99.03225806451613 97.90322580645162
0.7 99.49820788530467 98.06451612903226
0.7999999999999999 99.51612903225808 98.06451612903226
0.8999999999999999 99.51612903225808 98.2258064516129
0.9999999999999999 99.51612903225808 98.2258064516129
1.0999999999999999 99.51612903225808 98.06451612903226
1.1999999999999997 99.51612903225808 98.06451612903226
1.2999999999999998 99.51612903225808 98.06451612903226
1.4 99.55197132616487 98.06451612903226
1.4999999999999998 99.6415770609319 98.06451612903226
1.5999999999999996 99.6594982078853 98.06451612903226
1.6999999999999997 99.6774193548387 98.06451612903226
1.7999999999999998 99.69534050179212 98.06451612903226
1.8999999999999997 99.74910394265233 98.06451612903226


In [11]:
KERNEL = "RBF"
for GAMMA in [1e-5,1e-4,1e-3,1e-2,1e-1]:
    train_acc,test_acc = evaluate(dataset_split)
    print(GAMMA, train_acc,test_acc)

1e-05 50.94583333333333 50.075
0.0001 87.71805555555555 87.2625
0.001 91.85138888888889 91.0875
0.01 97.62499999999999 95.9375
0.1 100.0 96.8875


In [12]:
GAMMA = 0.5
train_acc,test_acc = evaluate(dataset_split)
print(GAMMA, train_acc,test_acc)

0.5 100.0 60.225


In [16]:
C = 0.5
GAMMA = 0.1
train_acc,test_acc = evaluate(dataset_split)
print(GAMMA, train_acc,test_acc)

0.1 99.19166666666666 96.425


In [22]:
KERNEL = "LINEAR"
df = pd.DataFrame()
for C in np.arange(0.1,2,0.2):
    train_acc,test_acc = evaluate(dataset_split)
    print(C,train_acc,test_acc)
    df = df.append({
        'train_acc':train_acc,
        'test_acc':test_acc
    },ignore_index=True)
df.to_csv('linear.csv',index=False)

0.1 93.00833333333334 91.4875
0.30000000000000004 93.28888888888889 91.825
0.5000000000000001 93.45 91.6875
0.7000000000000001 93.53194444444445 91.6375
0.9000000000000001 93.57777777777778 91.5875
1.1000000000000003 93.65833333333333 91.575
1.3000000000000003 93.69722222222222 91.625
1.5000000000000004 93.74305555555557 91.55
1.7000000000000004 93.75277777777777 91.6125
1.9000000000000004 93.77638888888887 91.6375


In [24]:
KERNEL = "POLY"
df = pd.DataFrame()
for C in np.arange(0.1,1,0.1):
    for POWER in [2,3,4]:
        for COEFF in [0,1,2,3]:
            train_acc,test_acc = evaluate(dataset_split)
            print(f'C: {C}, Power: {POWER}, COEFF: {COEFF}, train_acc = {train_acc}, test_acc = {test_acc}')
            df = df.append({
                'train_acc':train_acc,
                'test_acc':test_acc
            },ignore_index=True)
df.to_csv('poly.csv',index=False)

KeyboardInterrupt: 

In [11]:
KERNEL = "RBF"
C = 2.55
MODE = "LIBSVM"
bestgamma, bestacc = 0,0
for GAMMA in [1e-5,1e-4,1e-3,1e-2,1e-1]:
    train_acc,test_acc = evaluate(dataset_split)
    if test_acc>bestacc:
        bestacc=  test_acc
        bestgamma = GAMMA
    print(GAMMA, train_acc,test_acc)
print(bestgamma)

1e-05 12.45 12.45
0.0001 72.61666666666666 70.3
0.001 89.91666666666667 88.1
0.01 96.89444444444445 93.35
0.1 100.0 94.2
0.1
